# Traffic Light Detection and Classification - Udacity CarND Capstone
Using a pre-trained model to detect objects in an image.

In [1]:
import numpy as np
import os
import sys
import shutil
import tensorflow as tf
import time

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
%matplotlib inline

# Used for image processing and manipulations
if '/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path:
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

# Used for internal storage
import pickle

# Used for data augmentation
from skimage import exposure
from skimage import util

# Used for dividing the data into training and testing
import pandas as pd
from random import seed
from random import random

from glob import glob

/home/vision/anaconda3/envs/TF14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vision/anaconda3/envs/TF14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vision/anaconda3/envs/TF14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vision/anaconda3/envs/TF14/lib/python3.6/site-

## Object detection imports

Here are the imports from the object detection module.

In [2]:
PATH_TO_ROOT = r'/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/'
PATH_TO_OUTPUT = r'/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/trained_output/'
PATH_TO_API = r'/home/vision/saidz/tensorflow/models/research'

sys.path.append(PATH_TO_API)
sys.path.append(PATH_TO_API + '/object_detection')

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

## Model preparation

In [21]:
PATH_TO_LABELS = r'annotations/label_map.pbtxt'
NUM_CLASSES = 4

os.chdir(PATH_TO_ROOT)

M = "FRCNN" # More accurate labeling
M = "INCEPTION" # Fast labeling, but misses

if (M == "FRCNN"):
    try:
        shutil.rmtree('trained_output/final_frozen_frcnn')
    except OSError as e:
        print ("New frozen file will be generated in trained_output/final_frozen_frcnn.")

    !python /home/vision/saidz/tensorflow/models/research/object_detection/export_inference_graph.py \
        --pipeline_config_path=pretrained/annotation_config/faster_rcnn_resnet101_coco.config \
        --trained_checkpoint_prefix=training/model.ckpt-1000 \
        --output_directory=trained_output/final_frozen_frcnn/
    PATH_TO_MODEL = PATH_TO_ROOT + r'trained_output/final_frozen_frcnn/frozen_inference_graph.pb'
elif (M == "INCEPTION"):
    try:
        shutil.rmtree('trained_output/final_frozen_inception')
    except OSError as e:
        print ("New frozen file will be generated in pretrained/reextracted_frozen_inception.")
        
    !python /home/vision/saidz/tensorflow/models/research/object_detection/export_inference_graph.py \
        --pipeline_config_path=trained_output/extraction_config/ssd_inception_v2_coco.config \
        --trained_checkpoint_prefix=training/model.ckpt-5000 \
        --output_directory=trained_output/final_frozen_inception/
    PATH_TO_MODEL = PATH_TO_ROOT + r'trained_output/final_frozen_inception/frozen_inference_graph.pb'


/home/vision/anaconda3/envs/TF14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vision/anaconda3/envs/TF14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vision/anaconda3/envs/TF14/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vision/anaconda3/envs/TF14/lib/python3.6/site-

In [ ]:
python C:/Users/saidz/tensorflow/models/research/object_detection/export_inference_graph.py --pipeline_config_path=pretrained/train_config/faster_rcnn_resnet101_coco.config --trained_checkpoint_prefix=training/model.ckpt-614 --output_directory=trained_output/final_frozen_frcnn

In [10]:
PATH_TO_MODEL = PATH_TO_ROOT + r'trained_output/final_frozen_inception/frozen_inference_graph.pb'

In [12]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(category_index)

{1: {'id': 1, 'name': 'Red'}, 2: {'id': 2, 'name': 'Yellow'}, 3: {'id': 3, 'name': 'Green'}, 4: {'id': 4, 'name': 'None'}}


In [13]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [14]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_MODEL, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [15]:
print("detection_model.image_tensor: ")
print(detection_graph.get_tensor_by_name('image_tensor:0'), "\n")
print("detection_model.detection_boxes: ")
print(detection_graph.get_tensor_by_name('detection_boxes:0'), "\n")
print("detection_model.detection_scores: ")
print(detection_graph.get_tensor_by_name('detection_scores:0'), "\n")
print("detection_model.detection_classes: ")
print(detection_graph.get_tensor_by_name('detection_classes:0'), "\n")
print("detection_model.num_detections: ")
print(detection_graph.get_tensor_by_name('num_detections:0'), "\n")

detection_model.image_tensor: 
Tensor("image_tensor:0", shape=(?, ?, ?, 3), dtype=uint8) 

detection_model.detection_boxes: 
Tensor("detection_boxes:0", dtype=float32) 

detection_model.detection_scores: 
Tensor("detection_scores:0", dtype=float32) 

detection_model.detection_classes: 
Tensor("detection_classes:0", dtype=float32) 

detection_model.num_detections: 
Tensor("num_detections:0", dtype=float32) 



In [25]:
PATH_TO_TEST_IMAGES_DIR = 'C:/Users/saidz/git/TrafficLight_Detection-TensorFlowAPI/test_images_sim/'
PATH_TO_TEST_IMAGES_DIR = 'annotation_pipeline/raw-images-simulation/'


print(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*/*.jpg'))
TEST_IMAGE_PATHS = glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*/*.jpg'))

PATH_TO_TEST_IMAGES_DIR = PATH_TO_ROOT + 'annotation_pipeline/raw-images-simulation/'
TEST_IMAGE_PATHS = glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*/*.jpg'))

print("Length of test images:", len(TEST_IMAGE_PATHS))

annotation_pipeline/raw-images-simulation/*/*.jpg
Length of test images: 96


In [26]:
output_dict =[]
rows = len(TEST_IMAGE_PATHS) // 5 + 1
fig = plt.figure(figsize=(12,3*rows))

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        for icount in range(0,len(TEST_IMAGE_PATHS),1):
            print('Number {} out of {}, file name: {}'.format(icount, len(TEST_IMAGE_PATHS), TEST_IMAGE_PATHS[icount]))
            image_path = TEST_IMAGE_PATHS[icount]
            image = Image.open(image_path)

            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)

            time0 = time.time()

            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})

            time1 = time.time()

            boxes = np.squeeze(boxes)
            scores = np.squeeze(scores)
            classes = np.squeeze(classes).astype(np.int32)
            
            color = str(TEST_IMAGE_PATHS[icount]).replace('\\','/').split('/')[-2]

            output_dict.append({'path': str(TEST_IMAGE_PATHS[icount]).replace('\\','/'), 'color': color, \
                                'boxes': boxes, 'scores': scores, 'classes': classes})
            
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np, boxes, classes, scores,
                category_index,
                use_normalized_coordinates=True,
                line_thickness=6)

#             ax = plt.subplot(rows,5,len(output_dict))
#             ax.imshow(image_np)
#             ax.set_title("{}: {}".format(color, len(output_dict[icount]['boxes'])), fontsize=10)
#             ax.axis("off")
                    
# plt.show()

Number 0 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Yellow/left0012.jpg
Number 1 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Yellow/left0015.jpg
Number 2 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Yellow/left0022.jpg
Number 3 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Yellow/left0020.jpg
Number 4 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Yellow/left0580.jpg
Number 5 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Yellow/left0016.jpg
Number 6 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_

Number 54 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Green/left0545.jpg
Number 55 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Green/left0552.jpg
Number 56 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Green/left0556.jpg
Number 57 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Green/left0551.jpg
Number 58 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Green/left0558.jpg
Number 59 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Green/left0541.jpg
Number 60 out of 96, file name: /home/vision/saidz/git/CarND-Capstone/tl_detection

<Figure size 864x4320 with 0 Axes>

In [28]:
rows = len(output_dict) // 2 + 1
fig = plt.figure(figsize=(12,6*rows))
for i in range(len(output_dict)):
    print(output_dict[i]["path"],":")
    image = Image.open(output_dict[i]["path"])
    image_np = load_image_into_numpy_array(image)

    color = str(TEST_IMAGE_PATHS[i]).replace('\\','/').split('/')[-2]

    z=list(zip(output_dict[i]["classes"],output_dict[i]["scores"]))
    for j in range(len(z)):
        if (output_dict[i]["scores"][j]>0.2):
            print(color, "---", z[j])

'''
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np, output_dict[i]["boxes"], output_dict[i]["classes"], output_dict[i]["scores"],
        category_index,
        use_normalized_coordinates=True,
        min_score_thresh=.3,
        line_thickness=6)

        
    plt.figure(figsize=(6,6))
    plt.imshow(image_np)
    plt.axis("off")

    plt.show()
'''                    

/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Yellow/left0012.jpg :
Yellow --- (2, 0.5633654)
Yellow --- (2, 0.52695453)
/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Yellow/left0015.jpg :
Yellow --- (2, 0.73630255)
Yellow --- (2, 0.24856749)
Yellow --- (2, 0.21202809)
/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Yellow/left0022.jpg :
Yellow --- (2, 0.9178963)
Yellow --- (2, 0.7469897)
Yellow --- (2, 0.6343555)
/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Yellow/left0020.jpg :
Yellow --- (2, 0.76288617)
Yellow --- (2, 0.72310567)
Yellow --- (2, 0.5815188)
/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Yellow/left0580.jpg :
Yellow --- (2, 0.41161838)
/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/a

Green --- (3, 0.98426)
Green --- (3, 0.96937245)
Green --- (3, 0.96773636)
/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Green/left0544.jpg :
Green --- (3, 0.991497)
Green --- (3, 0.9734711)
Green --- (3, 0.921219)
/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Green/left0545.jpg :
Green --- (3, 0.991497)
Green --- (3, 0.9734711)
Green --- (3, 0.921219)
/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Green/left0552.jpg :
Green --- (3, 0.9790658)
Green --- (3, 0.9505286)
/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Green/left0556.jpg :
Green --- (3, 0.9825059)
Green --- (3, 0.9618094)
Green --- (3, 0.864406)
/home/vision/saidz/git/CarND-Capstone/tl_detection_pipeline/annotation_pipeline/raw-images-simulation/Green/left0551.jpg :
Green --- (3, 0.98200953)
Green --- (3, 

'\n    vis_util.visualize_boxes_and_labels_on_image_array(\n        image_np, output_dict[i]["boxes"], output_dict[i]["classes"], output_dict[i]["scores"],\n        category_index,\n        use_normalized_coordinates=True,\n        min_score_thresh=.3,\n        line_thickness=6)\n\n        \n    plt.figure(figsize=(6,6))\n    plt.imshow(image_np)\n    plt.axis("off")\n\n    plt.show()\n'

<Figure size 864x21168 with 0 Axes>